<a href="https://colab.research.google.com/github/salimaliraja/salimaliraja/blob/main/SalimAliraja_Classwork_with_streamlit_04112025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Class work:
#Base yourself on the multi-agent design principles to build a multi-agent linked-in blog writing system.

#The components/sub-agents will be:

#Web search
#Blog writer (that writes based on web search)
#SEO review and improvement
#You can use a different multi-agent design pattern. Remeber to use context to save data for intermittent steps in the workflow.

#Wrap it all up as a streamlit app

#https://docs.streamlit.io/develop/api-reference
"""
Streamlit Multi-Agent LinkedIn Blog Writer (FAISS + FactChecker + Versioning + Retries)

Features added:
- FactCheckerAgent: extracts claims and cross-references against retrieved sources using FAISS
- Vector store: FAISS local index for chunked + embedded web results
- Chunking and embedding using sentence-transformers
- Draft versioning and simple CMS stored in SQLite
- Rate-limit handling and retry logic using tenacity

Install requirements:
pip install streamlit requests openai transformers sentence-transformers faiss-cpu tenacity nltk

Notes:
- Provide API keys (OpenAI/SerpAPI/Bing) via sidebar or environment variables.
- This is a prototype; tune chunk sizes, embed model and LLM settings for production.

"""
!pip install streamlit
import streamlit as st
import sqlite3
import json
import time
import os
import requests
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass

# Retry/backoff
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

# Embeddings and FAISS
try:
    from sentence_transformers import SentenceTransformer
    import faiss
except Exception:
    SentenceTransformer = None
    faiss = None

# Optional LLM imports
try:
    import openai
except Exception:
    openai = None

# NLP helpers
import re

# ---------------------------
# Config & DB
# ---------------------------
DB_PATH = "rag_agents_context.db"
FAISS_INDEX_PATH = "faiss_index.bin"
EMBED_MODEL_NAME = os.environ.get("EMBED_MODEL", "sentence-transformers/all-MiniLM-L6-v2")
CHUNK_SIZE = 400  # characters per chunk (simple heuristic)
CHUNK_OVERLAP = 50

# ---------------------------
# Utilities
# ---------------------------

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    # context table
    cur.execute(
        """CREATE TABLE IF NOT EXISTS context (
               id TEXT PRIMARY KEY,
               payload TEXT,
               updated_at REAL
           )"""
    )
    # drafts table for CMS & versioning
    cur.execute(
        """CREATE TABLE IF NOT EXISTS drafts (
               draft_id TEXT,
               version INTEGER,
               title TEXT,
               content TEXT,
               metadata TEXT,
               created_at REAL,
               PRIMARY KEY(draft_id, version)
           )"""
    )
    conn.commit()
    conn.close()


def save_context(key: str, payload: Dict[str, Any]):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute(
        "REPLACE INTO context (id, payload, updated_at) VALUES (?, ?, ?)",
        (key, json.dumps(payload), time.time()),
    )
    conn.commit()
    conn.close()


def load_context(key: str) -> Optional[Dict[str, Any]]:
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT payload FROM context WHERE id = ?", (key,))
    row = cur.fetchone()
    conn.close()
    if row:
        return json.loads(row[0])
    return None

# Draft CMS helpers

def save_draft_version(draft_id: str, title: str, content: str, metadata: Dict[str, Any]):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT MAX(version) FROM drafts WHERE draft_id = ?", (draft_id,))
    r = cur.fetchone()
    max_v = r[0] if r and r[0] is not None else 0
    new_v = max_v + 1
    cur.execute(
        "INSERT INTO drafts (draft_id, version, title, content, metadata, created_at) VALUES (?, ?, ?, ?, ?, ?)",
        (draft_id, new_v, title, content, json.dumps(metadata), time.time()),
    )
    conn.commit()
    conn.close()
    return new_v


def list_drafts() -> List[Dict[str, Any]]:
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT draft_id, MAX(version), title, created_at FROM drafts GROUP BY draft_id")
    rows = cur.fetchall()
    conn.close()
    return [
        {"draft_id": r[0], "version": r[1], "title": r[2], "created_at": r[3]} for r in rows
    ]


def load_draft(draft_id: str, version: Optional[int] = None) -> Optional[Dict[str, Any]]:
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    if version is None:
        cur.execute("SELECT version, title, content, metadata, created_at FROM drafts WHERE draft_id = ? ORDER BY version DESC LIMIT 1", (draft_id,))
    else:
        cur.execute("SELECT version, title, content, metadata, created_at FROM drafts WHERE draft_id = ? AND version = ?", (draft_id, version))
    row = cur.fetchone()
    conn.close()
    if row:
        return {"version": row[0], "title": row[1], "content": row[2], "metadata": json.loads(row[3]) if row[3] else {}, "created_at": row[4]}
    return None

# ---------------------------
# Retry wrappers for HTTP & OpenAI
# ---------------------------

retry_decorator = retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=1, max=10), retry=retry_if_exception_type(Exception))

@retry_decorator
def safe_get(url, **kwargs):
    resp = requests.get(url, timeout=15, **kwargs)
    resp.raise_for_status()
    return resp

@retry_decorator
def safe_post(url, **kwargs):
    resp = requests.post(url, timeout=15, **kwargs)
    resp.raise_for_status()
    return resp

@retry_decorator
def openai_completion(prompt: str, max_tokens=700, engine="text-davinci-003", temperature=0.7):
    if not openai:
        raise RuntimeError("OpenAI package not available")
    resp = openai.Completion.create(engine=engine, prompt=prompt, max_tokens=max_tokens, temperature=temperature)
    return resp.choices[0].text.strip()

# ---------------------------
# Embedding & FAISS helpers
# ---------------------------

@dataclass
class Chunk:
    id: str
    text: str
    meta: Dict[str, Any]


class FaissStore:
    def __init__(self, embed_model_name: str = EMBED_MODEL_NAME, index_path: str = FAISS_INDEX_PATH):
        if SentenceTransformer is None or faiss is None:
            raise RuntimeError("Embedding model or faiss not installed. Install sentence-transformers and faiss-cpu")
        self.model = SentenceTransformer(embed_model_name)
        self.index_path = index_path
        self.index = None
        self.metadatas = []  # list of metadata dicts in same order as vectors
        self._load_index()

    def _load_index(self):
        if os.path.exists(self.index_path):
            try:
                self.index = faiss.read_index(self.index_path)
                # load metadatas
                meta_path = self.index_path + ".meta.json"
                if os.path.exists(meta_path):
                    with open(meta_path, "r", encoding="utf-8") as f:
                        self.metadatas = json.load(f)
                else:
                    self.metadatas = []
            except Exception:
                self.index = None
                self.metadatas = []
        if self.index is None:
            # initialize empty index (use cosine via normalized vectors)
            dim = self.model.get_sentence_embedding_dimension()
            self.index = faiss.IndexFlatIP(dim)
            self.metadatas = []

    def persist(self):
        faiss.write_index(self.index, self.index_path)
        with open(self.index_path + ".meta.json", "w", encoding="utf-8") as f:
            json.dump(self.metadatas, f, ensure_ascii=False)

    def add_chunks(self, chunks: List[Chunk]):
        texts = [c.text for c in chunks]
        embs = self.model.encode(texts, show_progress_bar=False)
        # normalize for cosine similarity
        import numpy as np
        embs = np.array(embs).astype("float32")
        faiss.normalize_L2(embs)
        self.index.add(embs)
        for c in chunks:
            self.metadatas.append(c.meta)
        self.persist()

    def query(self, query_text: str, top_k: int = 5) -> List[Tuple[Dict[str, Any], float]]:
        q_emb = self.model.encode([query_text])
        import numpy as np
        q_emb = np.array(q_emb).astype("float32")
        faiss.normalize_L2(q_emb)
        D, I = self.index.search(q_emb, top_k)
        results = []
        for score, idx in zip(D[0], I[0]):
            if idx < len(self.metadatas):
                results.append((self.metadatas[idx], float(score)))
        return results

# ---------------------------
# Agents
# ---------------------------

class Agent:
    def __init__(self, name: str):
        self.name = name

    def run(self, ctx: Dict[str, Any]) -> Dict[str, Any]:
        raise NotImplementedError


class WebSearchAgent(Agent):
    def __init__(self, serpapi_key: Optional[str] = None, bing_api_key: Optional[str] = None):
        super().__init__("web_search")
        self.serpapi_key = serpapi_key
        self.bing_api_key = bing_api_key

    def serpapi_search(self, query: str, num: int = 5) -> List[Dict[str, str]]:
        url = "https://serpapi.com/search.json"
        params = {"q": query, "engine": "google", "num": num, "api_key": self.serpapi_key}
        resp = safe_get(url, params=params)
        data = resp.json()
        results = []
        for r in data.get("organic_results", [])[:num]:
            results.append({"title": r.get("title"), "snippet": r.get("snippet"), "link": r.get("link"), "source": "serpapi"})
        return results

    def bing_search(self, query: str, num: int = 5) -> List[Dict[str, str]]:
        subscription_key = self.bing_api_key
        search_url = "https://api.bing.microsoft.com/v7.0/search"
        headers = {"Ocp-Apim-Subscription-Key": subscription_key}
        params = {"q": query, "count": num}
        resp = safe_get(search_url, headers=headers, params=params)
        data = resp.json()
        results = []
        for r in data.get("webPages", {}).get("value", [])[:num]:
            results.append({"title": r.get("name"), "snippet": r.get("snippet"), "link": r.get("url"), "source": "bing"})
        return results

    def run(self, ctx: Dict[str, Any]) -> Dict[str, Any]:
        query = ctx.get("topic", "")
        num = ctx.get("search_num", 5)
        results = []
        try:
            if self.serpapi_key:
                results = self.serpapi_search(query, num)
            elif self.bing_api_key:
                results = self.bing_search(query, num)
            else:
                results = ctx.get("manual_search", [])
        except Exception as e:
            ctx.setdefault("logs", []).append(f"WebSearchAgent error: {e}")
            results = ctx.get("manual_search", [])

        # store full html/snippet fetch optionally
        enhanced = []
        for r in results:
            # try to fetch link content briefly
            content = ""
            try:
                if r.get("link"):
                    resp = safe_get(r["link"]) if r["link"].startswith("http") else None
                    if resp:
                        content = resp.text[:2000]
            except Exception:
                content = ""
            enhanced.append({"title": r.get("title"), "snippet": r.get("snippet"), "link": r.get("link"), "content": content, "source": r.get("source")})

        ctx["search_results"] = enhanced
        ctx.setdefault("logs", []).append(f"WebSearchAgent: retrieved {len(enhanced)} items.")
        save_context("current_job", ctx)
        return ctx


class Chunker:
    @staticmethod
    def chunk_text(text: str, size: int = CHUNK_SIZE, overlap: int = CHUNK_OVERLAP) -> List[str]:
        if not text:
            return []
        chunks = []
        start = 0
        while start < len(text):
            end = min(start + size, len(text))
            chunk = text[start:end]
            chunks.append(chunk)
            start = end - overlap
            if start < 0:
                start = 0
            if start >= len(text):
                break
        return chunks


class BlogWriterAgent(Agent):
    def __init__(self, llm_backend: str = "openai", openai_key: Optional[str] = None, hf_model: Optional[str] = None):
        super().__init__("blog_writer")
        self.llm_backend = llm_backend
        self.openai_key = openai_key
        self.hf_model = hf_model
        if self.llm_backend == "openai" and self.openai_key and openai:
            openai.api_key = self.openai_key

    def _prompt_from_search(self, topic: str, results: List[Dict[str, Any]]) -> str:
        snippets = "\n\n".join([f"- {r.get('title','')} : {r.get('snippet','')}\nLink: {r.get('link','')}" for r in results])
        prompt = (
            f"Write a LinkedIn-style blog post (~500-800 words) about: {topic}.\n\n"
            "Use the following sources and weave them into a clear, engaging narrative with practical takeaways, headings, and a short conclusion.\n\n"
            f"Sources:\n{snippets}\n\n"
            "Style: professional, friendly, slightly conversational. Include an engaging hook, 3-5 short sections (with headings), "
            "practical tips, and a 1-2 line call-to-action at the end. Keep sentences concise for LinkedIn readers."
        )
        return prompt

    def _generate_openai(self, prompt: str, max_tokens: int = 700) -> str:
        return openai_completion(prompt, max_tokens=max_tokens)

    def run(self, ctx: Dict[str, Any]) -> Dict[str, Any]:
        topic = ctx.get("topic", "Untitled")
        results = ctx.get("search_results", [])
        prompt = self._prompt_from_search(topic, results)
        ctx["blog_prompt"] = prompt
        try:
            if self.llm_backend == "openai" and self.openai_key:
                blog = self._generate_openai(prompt)
            else:
                # fallback: simple template if no LLM available
                blog = f"## {topic}\n\nThis is a draft based on {len(results)} sources. Please provide a full LLM output here."
        except Exception as e:
            ctx.setdefault("logs", []).append(f"BlogWriterAgent error: {e}")
            blog = ctx.get("manual_draft", "## Draft\n\n(Provide manual draft here)")
        ctx["draft"] = blog
        ctx.setdefault("logs", []).append("BlogWriterAgent: draft generated.")
        save_context("current_job", ctx)
        return ctx


class SEOAgent(Agent):
    def __init__(self, llm_backend: str = "openai", openai_key: Optional[str] = None):
        super().__init__("seo_reviewer")
        self.llm_backend = llm_backend
        self.openai_key = openai_key
        if self.llm_backend == "openai" and self.openai_key and openai:
            openai.api_key = self.openai_key

    def _seo_prompt(self, draft: str, topic: str, keywords: List[str] = None) -> str:
        keys = ", ".join(keywords) if keywords else "None"
        prompt = (
            "You are an SEO expert and copywriter. Improve the following LinkedIn blog post for SEO and readability.\n\n"
            f"Topic: {topic}\n"
            f"Target keywords: {keys}\n\n"
            "Tasks:\n"
            "1) Suggest a catchy title (<= 12 words).\n"
            "2) Provide 5 meta description options (max 155 chars each).\n"
            "3) Propose 8 short, relevant hashtags.\n"
            "4) Provide an improved blog draft focusing on clarity, headings, and keyword inclusion. Keep LinkedIn tone.\n\n"
            "Original draft:\n" + draft + "\n\n"
            "Output in JSON with keys: title, meta_descriptions (list), hashtags (list), improved_draft."
        )
        return prompt

    def run(self, ctx: Dict[str, Any]) -> Dict[str, Any]:
        draft = ctx.get("draft", "")
        topic = ctx.get("topic", "")
        keywords = ctx.get("keywords", [])
        prompt = self._seo_prompt(draft, topic, keywords)
        ctx["seo_prompt"] = prompt
        try:
            raw = openai_completion(prompt, max_tokens=800) if (self.llm_backend == "openai" and self.openai_key) else ""
            parsed = None
            try:
                parsed = json.loads(raw)
            except Exception:
                # try extract JSON substring
                start = raw.find("{")
                end = raw.rfind("}") + 1
                if start != -1 and end != -1:
                    try:
                        parsed = json.loads(raw[start:end])
                    except Exception:
                        parsed = None
            if parsed:
                ctx["seo"] = parsed
                ctx.setdefault("logs", []).append("SEOAgent: parsed JSON output.")
            else:
                ctx.setdefault("logs", []).append("SEOAgent: couldn't parse JSON; saving raw output.")
                ctx["seo_raw"] = raw
        except Exception as e:
            ctx.setdefault("logs", []).append(f"SEOAgent error: {e}")
        save_context("current_job", ctx)
        return ctx


class FactCheckerAgent(Agent):
    """
    Extracts candidate claims from the draft and cross-checks them against the FAISS store of sources.
    It returns a list of claims with supporting evidence or flags as 'unverified'.
    """
    def __init__(self, faiss_store: FaissStore):
        super().__init__("fact_checker")
        self.store = faiss_store

    def extract_claims(self, text: str) -> List[str]:
        # naive claim extraction: sentences containing numbers, years, percentages, or 'research' keywords
        sentences = re.split(r'(?<=[.!?])\s+', text)
        claims = []
        for s in sentences:
            if re.search(r"\d|%|\byears?\b|study|research|survey|reported|found|according to", s, re.I):
                cleaned = s.strip()
                if len(cleaned) > 20:
                    claims.append(cleaned)
        # limit claims
        return claims[:20]

    def check_claim(self, claim: str, top_k: int = 5) -> Dict[str, Any]:
        # query FAISS for supporting sources
        results = self.store.query(claim, top_k=top_k)
        # if highest score > threshold -> supported
        if results and results[0][1] > 0.2:
            evidence = []
            for meta, score in results:
                evidence.append({"score": score, "source": meta.get("link"), "title": meta.get("title"), "snippet": meta.get("text")[:300]})
            return {"claim": claim, "status": "supported", "evidence": evidence}
        else:
            return {"claim": claim, "status": "unverified", "evidence": []}

    def run(self, ctx: Dict[str, Any]) -> Dict[str, Any]:
        draft = ctx.get("draft", "")
        claims = self.extract_claims(draft)
        checks = []
        for c in claims:
            checks.append(self.check_claim(c))
        ctx["fact_checks"] = checks
        ctx.setdefault("logs", []).append(f"FactCheckerAgent: checked {len(checks)} claims.")
        save_context("current_job", ctx)
        return ctx

# ---------------------------
# Orchestrator
# ---------------------------
class Orchestrator:
    def __init__(self, agents: List[Agent]):
        self.agents = agents

    def run(self, ctx: Dict[str, Any]) -> Dict[str, Any]:
        for agent in self.agents:
            ctx = agent.run(ctx)
        return ctx

# ---------------------------
# Streamlit UI
# ---------------------------
st.set_page_config(page_title="Multi-Agent LinkedIn Blog Writer (FAISS+FactCheck)", layout="wide")
st.title("Multi-Agent LinkedIn Blog Writer — FAISS + Fact-Check + CMS")

init_db()

with st.sidebar:
    st.header("Job & API Config")
    topic = st.text_input("Blog topic / prompt", value="How to build resilient teams in hybrid work")
    search_num = st.number_input("Search results to retrieve", min_value=1, max_value=20, value=5)
    st.markdown("**Web Search API keys (optional)**")
    serpapi_key = st.text_input("SerpAPI key", value="", type="password")
    bing_api_key = st.text_input("Bing API key", value="", type="password")
    st.markdown("**LLM settings**")
    llm_provider = st.selectbox("LLM backend", options=["openai", "local"], index=0)
    openai_key = st.text_input("OpenAI API key", value="", type="password") if llm_provider == "openai" else ""
    st.markdown("**Embedding model**")
    embed_model = st.text_input("Embedding model (sentence-transformers)", value=EMBED_MODEL_NAME)
    st.markdown("---")
    st.button("Save config", key="save_conf")

# inputs
st.header("Inputs & Manual Overrides")
col1, col2 = st.columns([1,1])
with col1:
    st.subheader("Manual search results (optional)")
    manual_search_json = st.text_area("Paste JSON list of {title,snippet,link,content} or leave blank", height=140)
    manual_search = []
    if manual_search_json.strip():
        try:
            manual_search = json.loads(manual_search_json)
        except Exception as e:
            st.error("Manual search JSON parse error: " + str(e))
with col2:
    st.subheader("Manual draft (optional fallback)")
    manual_draft = st.text_area("Paste a manual draft to use if LLM fails", height=140)

# session context
if "current_job" not in st.session_state:
    st.session_state.current_job = {"topic": topic, "search_num": search_num, "keywords": []}
else:
    st.session_state.current_job.update({"topic": topic, "search_num": search_num})

ctx = st.session_state.current_job
if manual_search:
    ctx["manual_search"] = manual_search
if manual_draft:
    ctx["manual_draft"] = manual_draft

# instantiate FAISS store
faiss_store = None
try:
    faiss_store = FaissStore(embed_model_name=embed_model)
    st.sidebar.success("FAISS store initialized")
except Exception as e:
    st.sidebar.warning("FAISS not available: " + str(e))

# Agents
web_agent = WebSearchAgent(serpapi_key=serpapi_key or None, bing_api_key=bing_api_key or None)
blog_agent = BlogWriterAgent(llm_backend=("openai" if llm_provider=="openai" else "local"), openai_key=openai_key or None)
seo_agent = SEOAgent(llm_backend=("openai" if llm_provider=="openai" else "local"), openai_key=openai_key or None)
fact_agent = FactCheckerAgent(faiss_store) if faiss_store else None

orch = Orchestrator([web_agent, blog_agent, seo_agent] + ([fact_agent] if fact_agent else []))

st.markdown("---")
if st.button("Run full pipeline"):
    with st.spinner("Running multi-agent pipeline..."):
        ctx = dict(ctx)
        ctx = orch.run(ctx)
        st.success("Pipeline finished.")
        save_context("current_job", ctx)
        st.session_state.current_job = ctx

if st.button("Run web search + index to FAISS"):
    with st.spinner("Searching and indexing to FAISS..."):
        ctx = web_agent.run(ctx)
        # chunk & index
        chunks = []
        for i, r in enumerate(ctx.get("search_results", [])):
            text = (r.get("title","") + "\n\n" + r.get("snippet","") + "\n\n" + r.get("content",""))
            parts = Chunker.chunk_text(text)
            for j, p in enumerate(parts):
                chunks.append(Chunk(id=f"{i}_{j}", text=p, meta={"title": r.get("title"), "link": r.get("link"), "text": p}))
        if faiss_store and chunks:
            faiss_store.add_chunks(chunks)
            st.success(f"Indexed {len(chunks)} chunks into FAISS.")
        else:
            st.warning("No FAISS store available or no chunks to index.")
        save_context("current_job", ctx)
        st.session_state.current_job = ctx

if st.button("Run blog writer"):
    with st.spinner("Running blog writer..."):
        ctx = blog_agent.run(ctx)
        st.success("Blog writer finished.")
        save_context("current_job", ctx)
        st.session_state.current_job = ctx

if st.button("Run SEO reviewer"):
    with st.spinner("Running SEO reviewer..."):
        ctx = seo_agent.run(ctx)
        st.success("SEO reviewer finished.")
        save_context("current_job", ctx)
        st.session_state.current_job = ctx

if st.button("Run Fact Checker") and fact_agent:
    with st.spinner("Running fact checker..."):
        ctx = fact_agent.run(ctx)
        st.success("Fact checking finished.")
        save_context("current_job", ctx)
        st.session_state.current_job = ctx

# Draft CMS actions
st.markdown("---")
st.header("Draft CMS & Versioning")
col1, col2 = st.columns([2,1])
with col1:
    draft_title = st.text_input("Draft title", value=ctx.get("topic","Untitled"))
    draft_edit = st.text_area("Edit draft", value=ctx.get("draft",""), height=300)
    if st.button("Save draft version"):
        draft_id = draft_title.lower().replace(" ", "_")
        metadata = {"saved_from_context": True}
        v = save_draft_version(draft_id, draft_title, draft_edit, metadata)
        st.success(f"Saved version {v} for draft {draft_id}")
with col2:
    st.subheader("Existing drafts")
    drafts = list_drafts()
    for d in drafts:
        st.write(f"- {d['draft_id']} (v{d['version']}) - {d['title']}")
    selected = st.text_input("Load draft_id (enter id)")
    ver = st.number_input("Version (0 = latest)", min_value=0, value=0)
    if st.button("Load draft") and selected:
        ld = load_draft(selected, None if ver==0 else int(ver))
        if ld:
            st.session_state.current_job.update({"draft": ld['content'], "topic": ld['title']})
            st.success(f"Loaded draft {selected} v{ld['version']}")
        else:
            st.error("Draft not found")

# Results display
st.markdown("---")
st.header("Results")
ctx_display = load_context("current_job") or ctx
if ctx_display.get("search_results"):
    st.subheader("Search results")
    for i, r in enumerate(ctx_display["search_results"], 1):
        st.markdown(f"**{i}. {r.get('title')}**")
        st.write(r.get('snippet'))
        st.write(r.get('link'))

if ctx_display.get("blog_prompt"):
    st.subheader("Blog prompt used")
    st.code(ctx_display["blog_prompt"])

if ctx_display.get("draft"):
    st.subheader("Generated Draft")
    st.write(ctx_display["draft"])

if ctx_display.get("seo"):
    st.subheader("SEO Output (parsed)")
    st.write(ctx_display["seo"])
elif ctx_display.get("seo_raw"):
    st.subheader("SEO Output (raw)")
    st.write(ctx_display["seo_raw"])

if ctx_display.get("fact_checks"):
    st.subheader("Fact Checks")
    for fc in ctx_display["fact_checks"]:
        st.markdown(f"**Claim:** {fc['claim']}")
        st.markdown(f"**Status:** {fc['status']}")
        if fc.get('evidence'):
            for e in fc['evidence']:
                st.write(f"- {e['title']} ({e['source']}) — score {e['score']:.3f}")

if ctx_display.get("logs"):
    st.subheader("Logs")
    for log in ctx_display['logs']:
        st.write('-', log)

st.markdown("---")
st.write("Persisted context in `rag_agents_context.db`. FAISS index in faiss_index.bin. Customize chunking, embedding, and LLM settings for better quality.")


2025-11-04 15:00:51.798 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 15:00:51.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 15:00:51.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 15:00:51.802 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 15:00:51.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 15:00:51.807 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 15:00:51.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 15:00:51.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar